In [1]:
# Импорт необходимых инструментов
from mnist import MNIST
import numpy as np
import matplotlib.pyplot as plt
import sys
mnist_data = MNIST('./MNIST_DATA')

In [2]:
def sig(current_value):
    return 1 / (1 + np.exp(-current_value))

def softmax(current_value):
    exp_val = np.exp(current_value - current_value.max())
    return exp_val / exp_val.sum()

nums = np.array([0, 1, 1, 0])

make_answer = np.vectorize(sig)
print(make_answer(nums))
print(softmax(nums))

[0.5        0.73105858 0.73105858 0.5       ]
[0.13447071 0.36552929 0.36552929 0.13447071]


In [52]:
# Скорость обучения
learing_speed = 0.3
# Длинна скрытого слоя
layer_length = 100
# Эпохи обучения
max_epochs = 10

In [53]:
images_train, labels_train = mnist_data.load_training()
learning_data = mnist_data.process_images_to_numpy(images_train)
correct_answers = mnist_data.process_labels(labels_train)

# Правильные ответы
y = correct_answers

In [54]:
# Входные нейроны
x = (learning_data / 255)

print(x)
print(np.shape(x))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(60000, 784)


In [55]:
# Веса 1 слоя
w_1 = np.random.normal(0, 0.1, (784, layer_length))

print(w_1)
print(np.shape(w_1))

# Веса 2 слоя
w_2 = np.random.normal(0, 0.1, (layer_length, 10))

print(w_2)
print(np.shape(w_2))

[[ 0.04918461  0.04761488  0.13585554 ...  0.00089948 -0.12779098
  -0.05314448]
 [ 0.05227146  0.1304075   0.2061333  ... -0.13302461  0.05159822
   0.10129407]
 [ 0.08246102  0.05996174 -0.08532035 ...  0.00289028  0.04104122
  -0.20263566]
 ...
 [-0.04449899  0.05078184  0.23120532 ... -0.12922203 -0.02854239
   0.01827026]
 [ 0.07298459  0.07637469  0.08001665 ... -0.06970336  0.06171578
   0.09766744]
 [ 0.11909714  0.07004941  0.09169188 ... -0.01381672 -0.02554213
   0.07473283]]
(784, 100)
[[-4.83165641e-03 -6.73787627e-02  1.30810672e-02 -1.16083774e-02
  -1.52120092e-02  6.18790072e-03  3.72763598e-03  1.11893991e-01
  -3.35787140e-02  2.20045925e-02]
 [-1.74143698e-01  5.88261232e-02 -8.82557385e-02  7.35178818e-02
   1.02567259e-01  3.79538103e-02  9.11492715e-02  1.11821717e-02
   1.91826888e-02 -8.56051880e-02]
 [ 3.75698445e-04 -9.53299758e-02  1.42375940e-02  6.10534965e-02
  -2.43977906e-01 -3.34824142e-02  1.66585515e-01  4.52273531e-02
  -4.18380945e-02  3.22671154e-

In [56]:
temp = np.eye(10)
make_answer_sig = np.vectorize(sig)

for epoch in range(max_epochs):
    for i in range(len(x)):
        y_1_1 = x[i]@w_1
        y_1_1 = make_answer_sig(y_1_1)

        y_1_2 = y_1_1@w_2
        y_1_2 = softmax(y_1_2)

        delta_2 = y_1_2 - temp[y[i]]
        dLi_dW2 = y_1_1.reshape((layer_length, 1))@delta_2.reshape((1,10))
        Ih = np.ones((1, layer_length))

        delta_1 = (y_1_1 * (Ih - y_1_1)) * (delta_2@w_2.T)
        dLi_dW1 = x[i].reshape((784, 1))@delta_1

        w_1 = w_1 - (learing_speed * dLi_dW1)
        w_2 = w_2 - (learing_speed * dLi_dW2)

In [57]:
y_1_1 = x@w_1
y_1_1 = make_answer_sig(y_1_1)
y_1_2 = y_1_1@w_2
y_1_2 = softmax(y_1_2)

# Выходные значения
print(y_1_2[0:20])

[[5.71478222e-24 1.18154054e-19 6.15412163e-19 4.40822001e-10
  2.37388383e-25 6.80796885e-08 2.19019682e-23 1.67802872e-19
  3.20695139e-19 1.78557675e-17]
 [8.67423889e-08 1.74616367e-22 1.87030949e-14 6.56844979e-20
  3.54413162e-22 1.13184198e-20 4.33281392e-20 8.18203975e-20
  1.57951612e-20 2.96788513e-17]
 [6.21958612e-21 3.89709054e-19 3.66048223e-18 2.06452723e-22
  2.02733656e-08 6.11407650e-21 1.17353959e-20 2.41632126e-16
  1.79877938e-20 1.66646405e-17]
 [3.82364338e-26 1.22543842e-06 1.05645520e-18 2.33767627e-21
  5.55765395e-15 2.05265590e-19 7.00360928e-20 6.09421174e-17
  1.26395422e-20 1.71618975e-20]
 [2.91246859e-19 1.66899416e-19 4.25608329e-21 2.52335504e-19
  2.35317804e-14 6.07886123e-19 2.59606055e-22 6.40769542e-19
  1.26319401e-18 1.49648272e-12]
 [1.48793926e-18 1.02767499e-18 1.46578177e-13 1.22931305e-17
  1.94154658e-19 2.21465955e-17 1.19450808e-19 2.11664515e-22
  2.33824705e-18 5.44892679e-20]
 [1.77716929e-21 8.73998939e-13 3.55458584e-19 3.72450289e

In [58]:
images_test, labels_test = mnist_data.load_testing()
learning_data = mnist_data.process_images_to_numpy(images_test)
correct_answers = mnist_data.process_labels(labels_test)

In [59]:
y = correct_answers
x = np.around(learning_data / 255)

# Тестовая выборка 10000 x 785
print(x)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
y_1_1 = x@w_1
y_1_1 = make_answer_sig(y_1_1)
y_1_2 = y_1_1@w_2
y_1_2 = softmax(y_1_2)

In [61]:
def predict(vector):
    return np.argmax(vector)

In [62]:
right = 0
for i in range(len(x)):
    if(predict(y_1_2[i]) == y[i]): right += 1

accyracy = right / 100

# Правильных ответов в %
print(accyracy)

94.14
